In [ ]:
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import netCDF4
import datetime
import geopandas as gpd
from shapely.geometry import box
import rioxarray
import rasterio
import regionmask
from shapely.geometry import mapping
import os,glob
import rasterio.plot
import cdsapi
from pyproj import CRS
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import statsmodels.api as sm


In [ ]:
### The AV Curve files shoulde be in the following format:
### Where Area is in km2 esm_mean is the mean volume in km3 from the ensemble of water 
### classification techniques, Temperature in deg Celcius, Precipitation is in mm, and TPI 
### is the terrain roughness. However you can change the code as per your need.

| Date | Area | esm_mean | esm_max | esm_min | Temperature | Precipitation | TPI |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 11/30/2022 | 0.026724797 | 1.17E-05 | 2.25E-05 | 0 | 24.437378 | 0 | 0.30 |

In [ ]:
data = []
for files in glob.glob(r'Path to all the AV curves files\\'+'*.csv'):
    f=pd.read_csv(files)
    f['Name'] = files.split('\\')[-1].split('Hyrdologic_AV_Curve_')[-1].split('.csv')[0] ### Adding a column of the name of lake to each AV file
    f['Date'] = pd.to_datetime(f['Date'])
    ### Considering the monthly cumulative precipitation for the regression
    f_monthly = f.groupby(f['Date'].dt.to_period('M')).sum()

    # Reset the index of the resulting DataFrame
    f_monthly.reset_index(inplace=True)

    # Rename the 'precipitation' column to 'monthly_precipitation'
    f_monthly.rename(columns={'precipitation': 'monthly_precipitation'}, inplace=True)

    # Merge the monthly precipitation data with the original DataFrame
    f_merged = pd.merge(f, f_monthly, left_on=f['Date'].dt.to_period('M'), right_on=f_monthly['Date'], how='left')

    # Drop the duplicate 'Date' column and unnecessary columns
    f_merged.drop(columns=['key_0', 'Date_y','esm_mean_y','esm_min_y','esm_max_y','Area_y','Height_y', 'temperature_y','TPI_y'], inplace=True)

    # # Rename the 'Date_x' column to 'Date'
    f_merged.rename(columns={'Date_x': 'Date','esm_max_x':'esm_max','esm_min_x':'esm_min','esm_mean_x':'esm_mean','Area_x':'Area','Height_x':'Height','temperature_x':'temperature','TPI_x':'TPI'}, inplace=True)

    # # Print the resulting DataFrame
    f_merged
    data.append(f_merged)

In [ ]:
### Combining all the AV Curves files irrespective of the regime together.
combined_df = pd.concat(data, ignore_index=True)

In [ ]:
combined_df['precipitation'] = combined_df['precipitation'].apply(lambda x: 0 if x <= 0 else x)
combined_df=combined_df.dropna().reset_index(drop = True)
combined_df['esm_mean (millionm3)']=combined_df['esm_mean']*1000 # Converting to km3
combined_df['esm_max (millionm3)']=combined_df['esm_max']*1000
combined_df['esm_min (millionm3)']=combined_df['esm_min']*1000
index_to_drop = []
for i in range(len(combined_df)):
    if 'Dighi' in combined_df['Name'][i]:  # If you want to drop a particular waterbody use this line other wise comment it.
        index_to_drop.append(i)
combined_df=combined_df.drop(index_to_drop).reset_index(drop = True)

In [ ]:
train_data, test_data = train_test_split(combined_df, test_size=0.2, random_state=47)

In [ ]:
train_data = train_data.reset_index(drop = True)
test_data = test_data.reset_index(drop = True)

In [ ]:
train_data = train_data.sort_values(by='Area')

In [ ]:
x = train_data[['Area','temperature','monthly_precipitation','TPI']]
y = train_data['esm_mean (millionm3)']
 
# with sklearn
regr = linear_model.LinearRegression()
regr.fit(x, y)

print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

In [ ]:
# with statsmodels
x = sm.add_constant(x) # adding a constant
 
model = sm.OLS(y, x).fit()
predictions = model.predict(x) 
 
print_model = model.summary()
print(print_model)

In [ ]:
train_data['pred_mean_million_m3'] = predictions

In [ ]:
train_data.head()

In [ ]:
from matplotlib.colors import ListedColormap
fig,ax = plt.subplots(figsize = (10,6))
# marker_shape = train_data['TPI'].astype('string')
# marker_shape = marker_shape.tolist() 
# colors = ['blue', 'green', 'red']
# cmap = ListedColormap(colors)
cmap='viridis'
ax.scatter(train_data['Area'],train_data['esm_mean (millionm3)'],color = 'black',label = 'Citizen Science & Satellite Based Estimated Volume')
ax.plot(train_data['Area'],train_data['pred_mean_million_m3'],color = 'blue',label = 'Regression Line')
ax.set_title('Hydrologic Regime based A-V Curves', fontname="Times New Roman", fontweight="bold",fontsize = 18)
ax.set_ylabel('Volume of Water Stored ($Million$ $M^3$)',fontname="Times New Roman", fontweight="bold",fontsize = 13)
ax.set_xlabel('Area of Water Body ($Km^2$)',fontname="Times New Roman", fontweight="bold",fontsize = 13)
ax.legend()
# textstr = '\n'.join((
#     r'$R^2=%.2f$' % (model.rsquared, ),
#     r'Volume of water body = Area x 2.5262 + Precipitaion x 0.0024 + Temperature x (-0.1307) + TPI x 3.8387 + 1.3692'
#     ))
# textstr = '\n'.join((
#     r'$R^2=%.2f$' % (model.rsquared, ),
#     '---------------------------------',
#     r'Volume depends on:', 'i) Area' ,
#        'ii) Precipitation' , 'iii) Temperature' ,'iv) TPI (Terrain Roughness)'
#     ))
textstr = '\n'.join((
    r'$R^2=%.2f$' % (model.rsquared, ),
    r'y = %.2f x Area %.2f x Temperature +' '\n'
    '%.3f x Monthly'' ' ' Precipitation + %.2f x TPI + %.2f ' % (model.params.Area, model.params.temperature,model.params.monthly_precipitation,model.params.TPI,model.params.const)
    ))
props = dict(boxstyle='round', facecolor='grey', alpha=0.1)
ax.text(0.020, 0.85, textstr, transform=ax.transAxes, fontsize=14,
        verticalalignment='top', bbox=props)